In [ ]:
# Define item names, get Workspace ID
# Use notebook utils to get WS id and save as variable, then get each item id after it is generated
thisworkspaceid = spark.conf.get("trident.workspace.id")
Name_Dataflow_EmptySalesTable = "CreateOrEmpty_Sales_Table"
Name_Dataflow_AppendSalesTable = "Append_Sales_Table"
Name_Dataflow_ReplaceDIMTables = "Replace_DIM_Tables"
Name_Pipeline_GenerateData = "Generate Data"

In [ ]:
# Create DF Market Lakehouse
import requests
import pandas as pd
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException

access_token = notebookutils.credentials.getToken("pbi")
headers = {"Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"}
url = f"https://api.fabric.microsoft.com/v1/workspaces/{thisworkspaceid}/lakehouses"
body = {
  "displayName": "DF_Market_LH",
  "description": "DF Market Lakehouse"
}
response = requests.post(url, headers=headers, json=body)
jsonresponse = response.json()
print(jsonresponse)
lakehouseid = jsonresponse['id']

In [ ]:
#Define Base64 Functions
import base64

def tobase64(textstring):
    textstring_bytes = textstring.encode("ascii")
    ascii_bytes = base64.b64encode(textstring_bytes)
    base64_output = ascii_bytes.decode("ascii")
    return base64_output

def frombase64(b64string):
    b64string_bytes = b64string.encode("ascii")
    ascii_bytes2 = base64.b64decode(b64string_bytes)
    text_output = ascii_bytes2.decode("ascii")
    return text_output

# Define Item from Definition Function
def CreateItemFromDefinition(wsid, itemname, itemtype, itemdefinition):
    access_token = notebookutils.credentials.getToken("pbi")
    headers = {"Authorization": f"Bearer {access_token}",
                "Content-Type": "application/json"}
    workspaceId = wsid     
    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items"
    body = {
        "displayName": itemname, 
        "type": itemtype, 
        "definition": itemdefinition
     }  
    response = requests.post(url, headers=headers, json = body)
    return response.json()

In [ ]:
# Create Dataflow - Empty Sales Table
import requests
import json5

url = "https://raw.githubusercontent.com/hoosierbi/fileshare/refs/heads/main/DFMarketFiles/V1/DataflowDefinition_EmptySalesTable.json"
itemdef = requests.get(url).text
itemdef = itemdef.replace('replacewithworkspaceid', thisworkspaceid)
itemdef = itemdef.replace('replacewithlakehouseid', lakehouseid)
itemdef = json5.loads(itemdef)
itemdef['parts'][0]['payload'] = tobase64(itemdef['parts'][0]['payload'])
itemdef['parts'][1]['payload'] = tobase64(itemdef['parts'][1]['payload'])
itemdef['parts'][2]['payload'] = tobase64(itemdef['parts'][2]['payload'])

createitem = CreateItemFromDefinition(thisworkspaceid, Name_Dataflow_EmptySalesTable, "Dataflow", itemdef)
print(createitem)
try:
    CreateSales_DataflowId = createitem['id']
except:
    print('fail')

In [ ]:
# Create Dataflow - Append Sales Table
import requests
import json5

url = "https://raw.githubusercontent.com/hoosierbi/fileshare/refs/heads/main/DFMarketFiles/V1/DataflowDefinition_AppendSalesTable.json"
itemdef = requests.get(url).text
itemdef = itemdef.replace('replacewithworkspaceid', thisworkspaceid)
itemdef = itemdef.replace('replacewithlakehouseid', lakehouseid)
itemdef = json5.loads(itemdef)
itemdef['parts'][0]['payload'] = tobase64(itemdef['parts'][0]['payload'])
itemdef['parts'][1]['payload'] = tobase64(itemdef['parts'][1]['payload'])
itemdef['parts'][2]['payload'] = tobase64(itemdef['parts'][2]['payload'])

createitem = CreateItemFromDefinition(thisworkspaceid, Name_Dataflow_AppendSalesTable, "Dataflow", itemdef)
print(createitem)
try:
    AppendSales_DataflowId = createitem['id']
except:
    print('fail')

In [ ]:
# Create Dataflow - DIM Tables
import requests
import json5

url = "https://raw.githubusercontent.com/hoosierbi/fileshare/refs/heads/main/DFMarketFiles/V1/DataflowDefinition_ReplaceDIMTables.json"
itemdef = requests.get(url).text
itemdef = itemdef.replace('replacewithworkspaceid', thisworkspaceid)
itemdef = itemdef.replace('replacewithlakehouseid', lakehouseid)
itemdef = json5.loads(itemdef)
itemdef['parts'][0]['payload'] = tobase64(itemdef['parts'][0]['payload'])
itemdef['parts'][1]['payload'] = tobase64(itemdef['parts'][1]['payload'])
itemdef['parts'][2]['payload'] = tobase64(itemdef['parts'][2]['payload'])
itemdef
createitem = CreateItemFromDefinition(thisworkspaceid, Name_Dataflow_ReplaceDIMTables, "Dataflow", itemdef)
print(createitem)
try:
    ReplaceDIMs_DataflowId = createitem['id']
except:
    print('fail')

In [ ]:
#Hard coded ids for troubleshooting
# lakehouseid = '3190cae1-4f77-45d5-bb3d-e5d60714e836'
# CreateSales_DataflowId = '5b9fbf44-7c9b-43bb-a63b-d8c29b0c806d'
# AppendSales_DataflowId = 'de68ac3e-7db6-4c98-9ae5-36c968dbab20'
# ReplaceDIMs_DataflowId = 'a72e870e-7aac-43b1-8bba-654f5800858f'

In [ ]:
# Create Pipeline - Generate Data
#Change DIMs dataflow to not use params for WS and LH IDs
import requests
import json5

url = "https://raw.githubusercontent.com/hoosierbi/fileshare/refs/heads/main/DFMarketFiles/V1/PipelineDefinition_GenerateData.json"
itemdef = requests.get(url).text
itemdef = itemdef.replace('replacewithworkspaceid', thisworkspaceid)
itemdef = itemdef.replace('replacewithlakehouseid', lakehouseid)
itemdef = itemdef.replace('CreateSalesDataflowId', CreateSales_DataflowId)
itemdef = itemdef.replace('ReplaceDIMsDataflowId', ReplaceDIMs_DataflowId)
itemdef = itemdef.replace('AppendSalesDataflowId', AppendSales_DataflowId)
itemdef = json5.loads(itemdef)
itemdef['parts'][0]['payload'] = tobase64(itemdef['parts'][0]['payload'])
itemdef['parts'][1]['payload'] = tobase64(itemdef['parts'][1]['payload'])
itemdef

createitem = CreateItemFromDefinition(thisworkspaceid, Name_Pipeline_GenerateData, "DataPipeline", itemdef)
createitem
# GenerateData_PipelineId = createitem['id']